In [1]:
import keras
from keras.layers import *
from keras.models import Model
from keras.models import model_from_json
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [2]:
def limit_mem():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

limit_mem()

In [3]:
def conv(ip, f, k=1, s=1, pad='same', d=1, use_bias=False):
    x = SeparableConv2D(f, k, strides=s, padding=pad, depth_multiplier=d, use_bias=use_bias)(ip)
    x = BatchNormalization()(x)
    x = ELU()(x)
    return x

In [4]:
def rconv(ip, f, k=1, s=1, pad='same', use_bias=False):
    x = Conv2D(f, k, strides=s, padding=pad, use_bias=use_bias)(ip)
    x = BatchNormalization()(x)
    x = ELU()(x)
    return x

In [5]:
ip_shp = (86,86, 1)

LABELS = list(range(10))

IMAGE_H, IMAGE_W = 86, 86
GRID_H,  GRID_W  = 3 , 3
BOX              = 1
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.3#0.5
NMS_THRESHOLD    = 0.3#0.45
# ANCHORS          = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]

NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [6]:
def create_model():
    ip = Input(ip_shp)

    x = rconv(ip, 16, k=3)
    x = rconv(x, 16, k=3)
    x = rconv(x, 16, k=3)
    x = rconv(x, 16, k=3, s=2, pad='valid')

    x = conv(x, 64, k=3)
    x = conv(x, 64, k=3)
    x = conv(x, 64, k=3)
    x = conv(x, 64, k=3, s=2, pad='valid')

    x = conv(x, 128, k=3)
    x = conv(x, 128, k=3)
    x = conv(x, 128, k=3)
    x = conv(x, 128, k=3, s=2, pad='valid')

    x = conv(x, 256, k=3)
    x = conv(x, 256, k=3)
    x = conv(x, 256, k=3)
    x = conv(x, 256, k=3, s=2, pad='valid')

    x = conv(x, 256, k=3)
    x = conv(x, 256, k=3)
    x = conv(x, 256, k=3)
    x = conv(x, 13, k=3)
    x = Conv2D(13, 3, strides=1, padding='valid', use_bias=True)(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('sigmoid')(x)

    return Model(ip, x)

model.summary()